# Dissociation of a molecule using the NEB method
https://wiki.fysik.dtu.dk/ase/tutorials/dissociation.html

In this tutorial we provide an illustrative example of a nudged-elastic band (NEB) calculation. For more information on the NEB technique, see `ase.neb`. We consider the dissociation of a nitrogen molecule on the Cu (111) surface.

The first step is to find the relaxed structures of the initial and final states.

In [1]:
from ase import Atoms
from ase.build import fcc111, add_adsorbate

In [2]:
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms

In [3]:
from ase.optimize import QuasiNewton

In [4]:
from ase.io import write

Find the initial and final states for the reaction.

Set up a (4 x 4) two layer slab of Cu:

In [5]:
slab = fcc111('Cu',size=(4,4,2))
slab.set_pbc((1,1,0))

Initial state. Add the N2 molecule oriented at 60 degrees:

In [6]:
d = 1.10 # N2 bond length
N2mol = Atoms('N2',positions=[[0.0,0.0,0.0],[0.5*3**0.5*d,0.5*d,0.0]])
add_adsorbate(slab,N2mol,height=1.0,position='fcc')

Use the EMT calculator for the forces and energies:

In [7]:
slab.set_calculator(EMT())

We don't want to worry about the Cu degrees of freedom, so fix these atoms:

In [8]:
mask = [atom.symbol == 'Cu' for atom in slab]
slab.set_constraint(FixAtoms(mask=mask))

Relax the structure

In [9]:
relax = QuasiNewton(slab)
relax.run(fmax=0.05)
print('initial state:', slab.get_potential_energy())
write('N2.traj', slab)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 09:55:16       14.459066*       9.8897
BFGSLineSearch:    1[  1] 09:55:17       12.244808*       4.3275
BFGSLineSearch:    2[  2] 09:55:17       11.343194*       2.1646
BFGSLineSearch:    3[  3] 09:55:17       11.077854*       0.6379
BFGSLineSearch:    4[  4] 09:55:17       11.056067*       0.2425
BFGSLineSearch:    5[  6] 09:55:18       11.049129*       0.2142
BFGSLineSearch:    6[  8] 09:55:18       11.034808*       0.1607
BFGSLineSearch:    7[  9] 09:55:18       11.032950*       0.0397
initial state: 11.032950397760606


Now the final state. Move the second N atom to a neighboring hollow site:

In [10]:
slab[-1].position[0] = slab[-2].position[0] + 0.25 * slab.cell[0,0]
slab[-1].position[1] = slab[-2].position[1]

and relax.

In [11]:
relax.run()
print('final state:  ', slab.get_potential_energy())
write('2N.traj', slab)

BFGSLineSearch:    8[ 11] 09:55:19       11.023212*       0.0971
BFGSLineSearch:    9[ 13] 09:55:19       11.021738*       0.1053
BFGSLineSearch:   10[ 15] 09:55:19       11.017702*       0.0733
BFGSLineSearch:   11[ 16] 09:55:20       11.017285*       0.0323
final state:   11.017284554452978


Having obtained these structures we set up an NEB calculation with 9 images. Using `interpolate()` provides a guess for the path between the initial and final states. We perform the relaxation of the images and obtain the intermediate steps.

In [12]:
import numpy as np

In [13]:
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.neb import NEB
from ase.optimize.fire import FIRE as QuasiNewton
from ase.io import read

Read the previous configurations

In [14]:
initial = read('N2.traj')
final = read('2N.traj')

Make 9 images (note the use of copy)

In [15]:
configs = [initial.copy() for i in range(8)] + [final]

As before, fix the Cu atoms

In [16]:
constraint = FixAtoms(mask=[atom.symbol != 'N' for atom in initial])
for config in configs:
    config.set_calculator(EMT())
    config.set_constraint(constraint)

Make the NEB object, interpolate to guess the intermediate steps

In [17]:
band = NEB(configs)
band.interpolate()

In [18]:
relax = QuasiNewton(band)

Do the calculation

In [19]:
relax.run()

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 09:55:22       11.066750*       0.5237
FIRE:    1 09:55:23       11.063989*       0.4622
FIRE:    2 09:55:24       11.059699*       0.3486
FIRE:    3 09:55:26       11.055742*       0.2000
FIRE:    4 09:55:27       11.053731*       0.0539
FIRE:    5 09:55:28       11.054185*       0.1269
FIRE:    6 09:55:30       11.054140*       0.1238
FIRE:    7 09:55:31       11.054053*       0.1175
FIRE:    8 09:55:32       11.053933*       0.1084
FIRE:    9 09:55:33       11.053791*       0.0968
FIRE:   10 09:55:35       11.053638*       0.0830
FIRE:   11 09:55:36       11.053488*       0.0678
FIRE:   12 09:55:37       11.053353*       0.0523
FIRE:   13 09:55:38       11.053230*       0.0393


True

Compare intermediate steps to initial energy

In [20]:
e0 = initial.get_potential_energy()
for config in configs:
    d = config[-2].position - config[-1].position
    print(np.linalg.norm(d), config.get_potential_energy() - e0)

1.6611276891332658 0.0
1.7387967149311636 0.0028614391665460204
1.8292016657795054 0.011524494980585942
1.9359265718620973 0.01860110017886285
2.0549987443423205 0.020279753673730028
2.183484274765332 0.01515099730038827
2.319173693283099 0.0038119114210726224
2.460281837433312 -0.009726934149639987
2.611961575822105 -0.015665843307628435


After the calculation is complete, the energy difference with respect to the initial state is given for each image, as well as the distance between the N atoms.